In [1]:
import pandas as pd
import re

In [2]:
df=pd.read_csv("data//data_original//1113_data.csv")
df_institution=pd.read_csv("data//item_department_match//institution_match.csv")
df_item=pd.read_csv("data//item_department_match//item_cleaned.csv")

In [3]:
df.loc[df["Institution_name"]=="Education.","Institution_name"]="Gauteng - education"

In [4]:
df["Institution_name"]=df["Institution_name"].apply(lambda x:str(x).upper())
df["Institution_name"]=df["Institution_name"].apply(lambda x:re.sub(r"\.","",x))
df_institution["Original_name"]=df_institution["Original_name"].apply(lambda x:x.upper())

In [5]:
df_clean_ins=pd.merge(df,df_institution,how="left",left_on="Institution_name",right_on="Original_name") 

In [6]:
df_use= pd.merge(df_clean_ins,df_item.loc[:,["Item_desp","category"]],how="left",on="Item_desp")
df_use.drop(columns=[" filter","Original_name"],inplace=True)

In [7]:
masks_cat=['PPE001 Surgical masks for patients','PPE002 Surgical masks for health care workers','Uncategorized masks','cloth masks',"respirator masks (FFP2, N95)"]

In [8]:
df_mask=df_use[df_use["category"].isin(masks_cat)]
df_mask.reset_index(inplace=True,drop=True)

In [9]:
df_mask.loc[~df_mask["Unit_price"].isna(),"Unit_price"]=df_mask.loc[~df_mask["Unit_price"].isna(),"Unit_price"].apply(lambda x: re.sub(",","",x))
df_mask.loc[:,"Unit_price"]=df_mask.loc[:,"Unit_price"].apply(lambda x: pd.to_numeric(x,errors="coerce"))

C:\Users\hanzh\anaconda3\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [10]:
df_mask.loc[:,"Quantity"]=df_mask.loc[:,"Quantity"].map(lambda x: re.sub(r'\D', '', str(x)))
df_mask.loc[:,"Quantity"]=df_mask.loc[:,"Quantity"].apply(lambda x: pd.to_numeric(x,errors="coerce"))

In [11]:
filter_numeric=(~df_mask["Quantity"].isna())&(df_mask["Unit_price"]>0)
filter_department=(~df_mask["Cleaned_name"].isna())
filter_amount=(df_mask["order_num"]>0)|(df_mask["pay_num"]>0)

In [12]:
df_mask.UOM.unique()

array(['Each', 'Box of 50', 'Other UOM', 'Box of 100', 'EA', '0', 'BOX',
       'Box of 10',
       'Function:Finance and Administration:Core Function:Finance: Default',
       'Each (Box of 10)', 'Surgical Face Mask with ties - 50 P/Box',
       'Pack of 10', 'Box 50', 'Pair', 'Box of 20', 'Pack of 50',
       'Each (Box of 50)', '1', 'Box of 100 gloves', '945', 'Per pair',
       'Surgical Face Mask - 50 P/Box', 'Surgical Face Mask - Box of 100',
       'per m2', '50 units per Box', '120', '210',
       'Cloth - Face mask - Emfuleni Logo', '3 Ply Surgical Mask', 'Unit',
       'BX',
       'Function:Public Safety:Core Function:Fire Fighting and Protection: Fire Fighting and Protection 6.3',
       'Function:Water Management:Core Function:Water Distribution',
       'Function:Waste Management:Core Function:Solid Waste Removal',
       '300', 'Pac', '5 Litre'], dtype=object)

In [13]:
unit_100=['Box of 100','Pack of 100','100 per box','BOX 100','100 in 1 box','1 box of 100s','100 in 1 box','(100 per pack)','100 pcs per box', '(100 per box )','(BOX100)','(A box of 100)']
unit_50=['Box of 50','Pack of 50','Box 50','Each (Box of 50)','50 units per Box','Pack of 50','(A box of 50)','50 in a box']
unit_10=['Box of 10','Each (Box of 10)','Pack of 10']
unit_20=['Box of 20','pack of 20']
unit_1=["Each","EA","1"]

In [14]:
df_mask.loc[df_mask["UOM"].isin(unit_100),"No_per_unit"]=100
df_mask.loc[df_mask["UOM"].isin(unit_50),"No_per_unit"]=50
df_mask.loc[df_mask["UOM"].isin(unit_10),"No_per_unit"]=10
df_mask.loc[df_mask["UOM"].isin(unit_20),"No_per_unit"]=20
df_mask.loc[df_mask["UOM"].isin(unit_1),"No_per_unit"]=1

C:\Users\hanzh\anaconda3\lib\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\hanzh\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [15]:
filter_unit=(~df_mask["No_per_unit"].isna())
filter_date=(~df_mask["Date_order"].str.contains("6484|8516|8517|8514",na=False))&(~df_mask["Date_payments"].str.contains("6484|8516|8517|8514",na=False))

In [16]:
df_mask_filter=df_mask[(filter_numeric)&(filter_unit)&(filter_department)&(filter_date)&(filter_amount)].copy()
df_mask_filter["price_per_mask"]=df_mask_filter["Unit_price"]/df_mask_filter["No_per_unit"]
df_mask_filter["real_quan"]=df_mask_filter["Quantity"]*df_mask_filter["No_per_unit"]
df_mask_filter.reset_index(inplace=True,drop=True)

In [17]:
filter_yaxis=(df_mask_filter["price_per_mask"]<=400)
filter_xaxis=(df_mask_filter["Date_order"].str.contains("2020"))
df_viz=df_mask_filter.loc[filter_yaxis&filter_xaxis,:].copy()

In [18]:
df_viz.loc[:,"Date_order"]=df_viz.loc[:,"Date_order"].apply(lambda x:pd.to_datetime(x, format='%d/%m/%Y'))
df_viz.loc[:,"Date_order"]=df_viz.loc[:,"Date_order"].dt.strftime("%Y-%m-%d")
df_viz.loc[:,"Date_payments"]=df_viz.loc[:,"Date_payments"].apply(lambda x:pd.to_datetime(x, format='%d/%m/%Y'))
df_viz.loc[:,"Date_payments"]=df_viz.loc[:,"Date_payments"].dt.strftime("%Y-%m-%d")

In [19]:
df_viz.rename(columns={"Item_desp":"Item_d","category":"Item_desp","Cleaned_name":"cleaned_name"},inplace=True)

In [20]:
df_viz.reset_index(inplace=True,drop=True)

In [21]:
df_viz.to_csv("data//data_viz//mask.csv")